In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_validate, ShuffleSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as XGBRegressor
from sklearn.model_selection import GridSearchCV
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
import pickle
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_pickle('df_model')

In [3]:
df = df.dropna(subset =['final_years', 'final_make'])
df =df.drop(['make','make_title', 'year', 'years', 'title_lower', 'URL', 'bikeTitle', 'attributes',], axis =1)
df = df[df['final_years'].astype(str).str.isdigit()]
df['odometer'] = df.odometer.str.extract('(^\d*)')
df[["final_years"]] = df[['final_years']].apply(pd.to_numeric)
df['Price']= df['Price'].apply(lambda x: x.replace('$',''))
df['Price']= df['Price'].apply(lambda x: x.replace(',',''))
df[['Price']] = df[['Price']].apply(pd.to_numeric)
df[['odometer']] = df[['odometer']].apply(pd.to_numeric)
df =df.reset_index(drop=True)
df = df[df['Price']<50000]
df=df[df['Price']>600]
df = df[df['odometer']<100000]
df = df[df['final_years']>1960]
df = df[df['final_years']<2022]
df.drop("VIN", inplace =True, axis =1)
df =df.reset_index(drop=True)
df['condition'] = df['condition'].str.replace('excellentcryptocurrency ok', 'excellent')
df['condition'] = df['condition'].str.replace('like newcryptocurrency ok', 'like new')
df['condition'] = df['condition'].str.replace('goodcryptocurrency ok', 'good')
df['condition'] = df['condition'].str.replace('faircryptocurrency ok', 'fair')
df['condition'] = df['condition'].str.replace('newcryptocurrency ok', 'new')

In [4]:
df['fuel'] = df['fuel'].fillna(value='other')
df['paint color'] = df['paint color'].fillna(value='other')
df['title status'] = df['title status'].fillna(value='other')
df['transmission'] = df['transmission'].fillna(value='other')
df['condition'] = df['condition'].fillna(value='other')
df['engine displacement (CC)'] = df['engine displacement (CC)'].fillna(value='other')
df['type'] = df['type'].fillna(value='other')

In [5]:
df = df[df['engine displacement (CC)']!='other']

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5045 entries, 3 to 12483
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Price                     5045 non-null   int64  
 1   title                     5045 non-null   object 
 2   fuel                      5045 non-null   object 
 3   odometer                  5045 non-null   float64
 4   paint color               5045 non-null   object 
 5   title status              5045 non-null   object 
 6   transmission              5045 non-null   object 
 7   condition                 5045 non-null   object 
 8   engine displacement (CC)  5045 non-null   object 
 9   type                      5045 non-null   object 
 10  final_make                5045 non-null   object 
 11  final_years               5045 non-null   int64  
dtypes: float64(1), int64(2), object(9)
memory usage: 512.4+ KB


In [7]:
# Use train_test_split to create training data and testing data
X = df.drop(['Price', 'title'], axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [8]:
categoricals = ['fuel','paint color','title status', 'transmission', 'condition', 'type', 'final_make','engine displacement (CC)']

In [9]:

pipeline1 = make_pipeline(
    ce.TargetEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestRegressor(max_depth=30,
                                      n_estimators=50, n_jobs=-3,
                                        random_state=30)
)
pipeline1.fit(X_train, y_train)

y_pred = pipeline1.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R^2:', r2_score(y_test, y_pred))
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

MAE: 1425.3235020375819
R^2: 0.7751828908554921
RMSE: 2193.4881115756566


In [10]:
pipeline1.fit(X,y)

Pipeline(steps=[('targetencoder',
                 TargetEncoder(cols=['fuel', 'paint color', 'title status',
                                     'transmission', 'condition',
                                     'engine displacement (CC)', 'type',
                                     'final_make'])),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=30, n_estimators=50, n_jobs=-3,
                                       random_state=30))])

In [12]:
with open("model_1.pkl", 'wb') as f:
    pickle.dump(pipeline1, f)

In [34]:
pipeline2 = make_pipeline(
    ce.TargetEncoder(),
    SimpleImputer(strategy='median'),
    XGBRegressor())

pipeline2.fit(X_train, y_train)
y_pred = pipeline2.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R^2:', r2_score(y_test, y_pred))
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

MAE: 1466.273978558282
R^2: 0.7516544315391219
RMSE: 2305.413398118557


In [35]:
pipeline4 = make_pipeline(
    ce.TargetEncoder(),
    SimpleImputer(strategy='median'),
    XGBRegressor(learning_rate=0.1,
                max_depth=6,
                min_child_weight=2,
                n_estimators =100,
                subsample=0.9))

pipeline4.fit(X_train, y_train)
y_pred = pipeline4.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R^2:', r2_score(y_test, y_pred))
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

MAE: 1470.675428145661
R^2: 0.7747478018553547
RMSE: 2195.609616594203


In [36]:
param_grid = {
    'learning_rate': [0.1, 0.5, 1],
    'max_depth': [2,5,6,10],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.5, 0.7, .9],
    'n_estimators': [10, 50, 100, 500 ],
}

In [37]:
clf =XGBRegressor()

In [38]:
grid_clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=1)


In [39]:
pipeline3 = make_pipeline(
    ce.TargetEncoder(),
    SimpleImputer(strategy='median'),
    grid_clf)

In [40]:
pipeline3.fit(X_train, y_train)
y_pred = pipeline3.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('R^2:', r2_score(y_test, y_pred))
print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

MAE: 1472.0230528140785
R^2: 0.779550457677505
RMSE: 2172.0769462757185


In [41]:
best_parameters = grid_clf.best_params_

print('Grid Search found the following optimal parameters: ')
for param_name in sorted(best_parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

Grid Search found the following optimal parameters: 
learning_rate: 0.1
max_depth: 6
min_child_weight: 3
n_estimators: 100
subsample: 0.9


In [42]:
pipeline4.fit(X,y)

Pipeline(steps=[('targetencoder',
                 TargetEncoder(cols=['fuel', 'paint color', 'title status',
                                     'transmission', 'condition',
                                     'engine displacement (CC)', 'type',
                                     'final_make'])),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.1,
                              max_delta_step=0, max_depth=6, min_child_weight=2,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=8, num_parallel_tree=1,
                             

In [22]:
with open("model.pkl", 'wb') as f:
    pickle.dump(pipeline4, f)